# Boston Airbnb
Using 2016 Airbnb data from the Boston area, I chose to explore metrics to help investors answer potential questions when comparing property locations. The data consists of 3306 Airbnb listings across 18 neighborhoods.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file = 'listings.csv'

original_df = pd.read_csv(file,index_col=0)

#Limit to potential columns of interest
df = original_df[['name','summary','description','transit','neighbourhood_cleansed','zipcode',
         'latitude','longitude','room_type','accommodates','bathrooms',	'bedrooms','beds',
         'amenities','price','guests_included','availability_30',	
         'availability_60','availability_90','availability_365','number_of_reviews',
         'first_review','review_scores_rating','reviews_per_month']]

In [ ]:
### Dealing with NAN ###
df.isnull().mean()

## Cleaning Data
Lets standardize the availaiblity columns and convert them to be Utilization Rate

In [ ]:
#Look at the data types in the dataset
df.dtypes

In [ ]:
#notice that price is an object rather than a float
df['price'].head()

In [ ]:
#remove the '$'' and the ',' price
def remove_sign(col):
    col = col.replace('$','')
    col = col.replace(',','')
    return float(col)

df['price'] = df['price'].apply(remove_sign)
df['price'].head()

In [ ]:
#standardizing availability and covert to Utilization Rate
df['utilization_30'] = df['availability_30'].apply(lambda x: 1 - (x/30))
df['utilization_60'] = df['availability_60'].apply(lambda x: 1 - (x/60))
df['utilization_90'] = df['availability_90'].apply(lambda x: 1 - (x/90))
df['utilization_365'] = df['availability_365'].apply(lambda x: 1 - (x/365))

In [ ]:
#how many Airbnbs listings are per neighborhood? Do we need to remove any that are too small of a sample?
df['neighbourhood_cleansed'].value_counts().sort_values()

In [ ]:
for nb, group in df.groupby('neighbourhood_cleansed'):
    count = len(group)
    df.loc[group.index,'count'] = count
    
#Filter out based off your own sample threshold. Original df has 3565 observations
#For this analysis, I chose to only look at neighborhoods with atleast 50 listings.
over_df = df[df['count'] >= 50] #input the minimum sample size

In [ ]:
#plot a box and whiskers to see price distributions
prices = over_df['price']
plt.boxplot(prices)

In [ ]:
#lets get rid of the extreme outliers
max_range = over_df['price'].quantile(.97)
max_range

In [ ]:
over_df = over_df[over_df['price'] <= max_range]
#only lost around 3% of the data when droppinng extreme outliers
#the other outliers will be dealt with by using median as the statistic of interest
over_df.shape


## Neighbourhood Analysis
Lets compare neighbourhoods using some metrics from the dataset

### 1. What is the Relationship Between Price and Utilization?

In [ ]:
#standardize price = Price/Max Price
max_price = over_df['price'].max()

over_df['std_price'] = over_df['price'].apply(lambda col: col/max_price)

In [ ]:
group_df = over_df.groupby('neighbourhood_cleansed')[['std_price','utilization_365']].median().sort_values(by='std_price')

In [ ]:
group_df.columns = ['Median Price','Utilization Rate']

group_df.plot(kind='barh',color=['tomato','gray'])
plt.xlabel('Percent of Max')
plt.ylabel('')
plt.title('What is the Relationship Between Price and Utilization Rate?')

### 2. What are the top 10 neighbourhoods with the highest median projected revenue?

In [ ]:
def plot_colors(df,sym,threshold):
    """ Pass through the top 10 df and the threshold to output the colors and the emphasis on the plots.
    df - dataframe
    sym - input either '>' if greater or '<' if less than the threshold
    threshold - top X percente to emphasize in the graph. Pass a decimal"""
    
    colors = []

    for i in df:
        if sym == '>':
            if i/df.max() >= threshold:
                colors.append('tomato')
            else:
                colors.append('lightsalmon')
        elif sym == '<':
            if i/df.max() <= threshold:
                colors.append('tomato')
            else:
                colors.append('lightsalmon')
            
    return colors

In [ ]:
#Revenue Projection == (Price * 365) * Yearly Utilization Rate
over_df['revenue_projection'] = (over_df['price']*365) * over_df['utilization_365']

In [ ]:
#get top 10 neighborhoods with highest revenue projections
top_10_rev = over_df.groupby('neighbourhood_cleansed')['revenue_projection'].median()
top_10_rev = top_10_rev.sort_values(ascending=False).iloc[0:10]

In [ ]:
#add specific graph coloring to emphasize the highest projected revenue
colors = plot_colors(top_10_rev,'>',.9)

#Graph Edits
top_10_rev.plot(kind='barh',legend=None,color=colors)
plt.xlabel('Median Revenue in a Year')
plt.ylabel('')
plt.title('Top 10 Boston Neighbourhoods with the Highest Project Revenues on Airbnb')

### 3. Which Neighborhoods Will Make Me My Money Back the Soonest?

In [ ]:
#Median house prices from https://www.redfin.com/
median_house_prices = {
    'Leather District':979500,
    'Fenway':1064500,
    'Roslindale':699000,
    'Dorchester':690000,
    'West Roxbury':750000,
    'Roxbury':739900,
    'Mattapan':610000,
    'Hyde Park':615000,
    'Bay Village':2878750,
    'East Boston':675000,
    'Brighton':500000,
    'Jamaica Plain':757500,
    'Mission Hill':865000,
    'Allston':539000,
    'South Boston':825000,
    'Longwood Medical Area':1200000,
    'Charlestown':952500,
    'North End':898000,
    'West End':518375,
    'Back Bay':1100000,
    'Downtown':1125000,
    'South End':1100100,
    'Beacon Hill':1200000,
    'Chinatown':1310000,
    'South Boston Waterfront':1134000
    }

In [ ]:
#use the median house prices to calculate years for ROI
#Years for ROI == Median House Price / Median Revenue Projection
for nb,group in over_df.groupby('neighbourhood_cleansed'):
    try:
        price = median_house_prices[nb]
        median_rev = group['revenue_projection'].median()
        over_df.loc[group.index,'yrs_roi'] = price/median_rev
    except:
        pass

In [ ]:
#get the top 10 neighborhoods with the lowest time for ROI
roi_df = over_df.groupby('neighbourhood_cleansed')['yrs_roi'].mean()
top_10_roi = roi_df.sort_values().iloc[0:10]

In [ ]:
colors = plot_colors(top_10_roi,'<',.85)

top_10_roi.plot(kind='barh',legend=None,color=colors)
plt.xlabel('Years for ROI')
plt.ylabel('')
plt.title('Top 10 Boston Neighbourhoods with the Shortest Time for ROI')